# UR5 Simulation Direct Kinematics
**Author** Eduardo Gomez

Using the Denavit-Hartenberg parameters of the UR5, program the direct Kinematics

In [13]:
# Import libraries
import zmqRemoteApi
import numpy as np
from time import sleep
from pathlib import Path
import pandas as pd

## Setup Data

Here we will set up the data for the Denavit-Hartenberg parameters. The data was from: ...

In [15]:
df = pd.read_csv("../data/ur5-kinematics.csv")
print(df.head())

   Kinematics    theta_[rad]    a_[m]      d_[m]     alpha_[rad]
0  Joint_1                  0     0.0000    0.1625          1.57
1  Joint_2                  0    -0.4250    0.0000          0.00
2  Joint_3                  0    -0.3922    0.0000          0.00
3  Joint_4                  0     0.0000    0.1333          1.57
4  Joint_5                  0     0.0000    0.0997         -1.57


The matriz transformations can be expressed like these: 

$$
H_i^{i-1} = Rot_z(\theta_i)T(0,0,d_i)T(a_i, 0, 0)Rotx(\alpha_i)
$$

Esto se lee como una rotacion $\theta$ en $z$, una translacion $d$ en $z$, translacion $a$ en $x$ y rotacion $\alpha$ en $x$,

## Setup simulation

To set up the simulation, open the `./sim/ur5-simulation.ttt` file. Then run the following code: 

> If there is an error, either the file is not opened or another script is controlling it

In [12]:
# Create a client to get connected to the zmqremoteApi sernver from CoppeliaSim
client = zmqRemoteApi.RemoteAPIClient()

# Get the remote object 'sim'
sim = client.getObject('sim')


Here, we set up the id for controlling the robot

In [3]:
UR5_sim_obj_id = "/UR5"
base_id = sim.getObject(UR5_sim_obj_id)

# the last joint is the rotor for the end effector
# does not affect the final position
no_joints = 6
joint_sim_names = [None for _ in range(6)]

joint_sim_names[0] = f"{UR5_sim_obj_id}/joint"

for i in range(1,len(joint_sim_names)):
    link_joint_str = "/link/joint" * i
    joint_sim_names[i] = f"{UR5_sim_obj_id}{link_joint_str}"

connection_name = f"{UR5_sim_obj_id}/connection"

In [4]:
# Assert that names have id. Error is thrown if id doesnt exists
joint_id = [
    sim.getObject(name) for name in joint_sim_names
]
connection_id = sim.getObject(connection_name)

In [8]:
def reset_arm():
    """
    Resets the arm position to 0 deg
    """
    for j in joint_id:
        sim.setJointTargetPosition(j, np.deg2rad(0))

In [ ]:
Here we start the simulation

In [6]:
sim.startSimulation()

1

In [9]:
reset_arm()

In [10]:

print(sim.getObjectPosition(connection_id, sim.handle_world))
angles = [90, -45, 80, -40, 100, 80]
angles = [np.deg2rad(a) for a in angles]
assert len(angles) == no_joints

for a, j in zip(angles, joint_id):
    sim.setJointTargetPosition(j, a)

sleep(1)
print("Connection final position")
print(sim.getObjectPosition(connection_id, sim.handle_world))



[-0.38503810662955174, 0.22499981038377784, 1.0011865280398988]
Connection final position
[-0.042423557235479054, 0.12796914759588124, 0.7989468163579077]


Here we stop the simulation

In [11]:
sim.stopSimulation()

1